In [1]:
pip install spacy==2.3.5


Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy-2.3.5-cp39-cp39-manylinux2014_x86_64.whl (10.3 MB)
  Using cached thinc-7.4.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz#egg=en_core_web_sm==2.3.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 292.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=7d1e28a27361aa5d563151c26dc5614dceec8a5f34fb3742fa930b08d0f18548
  Stored in directory: /tmp/pip-ephem-wheel-cache-g48yehaf/wheels/19/d6/1c/5484b95647df5d7afaf74abde458c66c1cd427e69e801fe826
Successfully built en_core_web_sm
  Attempting uninstall: en_core_web_sm
    Found existing installati

In [5]:
import en_core_web_sm
nlp = en_core_web_sm.load()

OSError: [E053] Could not read config file from /home/pdesai6/.local/lib/python3.9/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg

In [2]:

import spacy
nlp = spacy.load("en_core_web_sm")

def mask_gender_bias(sentence):
    doc = nlp(sentence)

    # Gender neutral words that needs to be masked 
    gendered_words = {
        " he": "[MASK]",
        "she": "[MASK]",
        "her": "[MASK]",
        "hers": "[MASK]",
        "him": "[MASK]",
        "his": "[MASK]",
        "father": "parent",
        "mother": "parent",
        "brother": "sibling",
        "sister": "sibling",
        "men": "people",
        "women": "people",
        "guy": "person",
        "man": "person",
        "woman": "person",
        "boy": "child",
        "boys": "children",
        "girls": "children",
        "girl": "child"
    }
    
    for token in doc:
        if token.text.lower() in gendered_words:
            sentence = sentence.replace(token.text, gendered_words[token.text.lower()])
        #Masking the names
        if token.ent_type_ == "PERSON":
            sentence = sentence.replace(token.text, "[MASK]")

    return sentence



In [11]:
sentence = "John does not want to go to the. His father is worried about him"
masked_sentence = mask_gender_bias(sentence)
print(sentence)
print(masked_sentence)  


John does not want to go to the. His father is worried about him
[MASK] does not want to go to the. [MASK] parent is worried about [MASK]


In [8]:
#running it on our dataset
import json

with open("english.json") as f:
    data = json.load(f)

for sentence_dict in data:
    for key in ["stereotype", "anti-stereotype"]:
        sentence_dict[key] = mask_gender_bias(sentence_dict[key])

with open("english_masked.json", "w") as f:
    json.dump(data, f, indent=4)
